In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary
import public_tests

%matplotlib inline

## A. Identity block

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
    <caption><center> <u> <font color='purple'> <b>Figure 4</b> </u><font color='purple'>  : <b>Identity block.</b> Skip connection "skips over" 3 layers.</center></caption>

In [2]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):

    F1, F2, F3 = filters

    X_shortcut = X

    layer = layers.Conv2D(F1, kernel_size=1, strides=1, padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut) # F1 bo loc
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    layer = layers.Activation('relu')(layer)

    layer = layers.Conv2D(F2, kernel_size=f, strides=1, padding='same', kernel_initializer=initializer(seed=0))(layer) # F1 bo loc
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    layer = layers.Activation('relu')(layer)

    layer = layers.Conv2D(F3, kernel_size=1, strides=1, padding='valid', kernel_initializer=initializer(seed=0))(layer) # F1 bo loc
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    
    layer = layers.Add()([layer, X_shortcut])
    layer = layers.Activation('relu')(layer)

    return layer

In [3]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

public_tests.identity_block_test(identity_block)

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71236 192.71236 192.71236  96.85618]
  [ 96.85618  96.85618  96.85618  48.9281 ]]

 [[578.13715 578.13715 578.13715 290.56854]
  [290.56854 290.56854 290.56854 146.78427]]]
96.85618

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
All tests passed!


## B. Convolutional block

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 4</b> </u><font color='purple'>  : <b>Convolutional block</b> </center></caption>

In [4]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    
    F1, F2, F3 = filters
    X_shortcut = X

    layer = layers.Conv2D(F1, kernel_size=1, strides=s, padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    layer = layers.Activation('relu')(layer)

    layer = layers.Conv2D(F2, kernel_size=f, strides=1, padding='same', kernel_initializer=initializer(seed=0))(layer)
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    layer = layers.Activation('relu')(layer)

    layer = layers.Conv2D(F3, kernel_size=1, strides=1, padding='valid', kernel_initializer=initializer(seed=0))(layer)
    layer = layers.BatchNormalization(axis=3)(layer, training=training)
    
    layer_ = layers.Conv2D(F3, kernel_size=1, strides=s, padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    layer_ = layers.BatchNormalization(axis=3)(layer_, training=training)

    layer = layers.Add()([layer, layer_])
    layer = layers.Activation('relu')(layer)

    return layer


In [5]:
from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')


tf.Tensor(
[[[0.         0.66683817 0.         0.         0.888539   0.5274254 ]
  [0.         0.65053666 0.         0.         0.8959285  0.49965227]]

 [[0.         0.6312079  0.         0.         0.86362475 0.47643146]
  [0.         0.56883204 0.         0.         0.8553412  0.417093  ]]], shape=(2, 2, 6), dtype=float32)
All tests passed!


## C. ResNet-50

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> <b>Figure 5</b> </u><font color='purple'>  : <b>ResNet-50 model</b> </center></caption>

In [6]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    inputs = layers.Input(input_shape)

    layer = layers.ZeroPadding2D((3, 3))(inputs)

    layer = layers.Conv2D(64, kernel_size=7, strides=2)(layer)
    layer = layers.BatchNormalization(axis=3)(layer)
    layer = layers.Activation('relu')(layer)
    layer = layers.MaxPool2D(pool_size=3, strides=2)(layer)

    layer = convolutional_block(layer, f=3, filters=[64, 64, 256], s=1)
    layer = identity_block(layer, f=3, filters=[64, 64, 256])
    layer = identity_block(layer, f=3, filters=[64, 64, 256])

    layer = convolutional_block(layer, f=3, filters=[128, 128, 512], s=2)
    layer = identity_block(layer, f=3, filters=[128, 128, 512])
    layer = identity_block(layer, f=3, filters=[128, 128, 512])
    layer = identity_block(layer, f=3, filters=[128, 128, 512])

    layer = convolutional_block(layer, f=3, filters=[256, 256, 1024], s=2)
    layer = identity_block(layer, f=3, filters=[256, 256, 1024])
    layer = identity_block(layer, f=3, filters=[256, 256, 1024])
    layer = identity_block(layer, f=3, filters=[256, 256, 1024])
    layer = identity_block(layer, f=3, filters=[256, 256, 1024])
    layer = identity_block(layer, f=3, filters=[256, 256, 1024])

    layer = convolutional_block(layer, f=3, filters=[512, 512, 2048], s=2)
    layer = identity_block(layer, f=3, filters=[512, 512, 2048])
    layer = identity_block(layer, f=3, filters=[512, 512, 2048])

    layer = layers.AveragePooling2D(pool_size=2)(layer)
    layer = layers.Flatten()(layer)
    outputs = layers.Dense(classes, activation='softmax')(layer)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [7]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

In [8]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 70, 70, 3) │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 32, 32,    │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 15, 15,    │          0 │ activation_18[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 15, 15,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 15, 15,    │     36,928 │ activation_19[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 15, 15,    │     16,640 │ activation_20[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 15, 15,    │     16,640 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_23[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_24[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 23,600,006 (90.03 MB)

 Trainable params: 23,546,886 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))

X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)


In [11]:
model.fit(X_train, Y_train, epochs=20, batch_size=32)

Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.3527 - loss: 2.7184
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.6532 - loss: 1.2070
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.6955 - loss: 0.8814
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.8774 - loss: 0.3535
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.9290 - loss: 0.2001
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9298 - loss: 0.2195
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9607 - loss: 0.1013
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9653 - loss: 0.1827
Epoch 9/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9196 - loss: 0.2634
Epoch 10/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9234 - loss: 0.2266
Epoch 11/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9440 - loss: 0.1848
Epoch 12/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9629 - lo

In [12]:
model.evaluate(X_test, Y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.9483 - loss: 0.2289


[0.2981865704059601, 0.9333333373069763]

In [16]:
model.get_metrics_result()

{'accuracy': 0.9333333373069763, 'loss': 0.2981865704059601}